## Documentation for Mongo 5.0 and Mongosh 1.6
https://www.mongodb.com/docs/v5.0/reference/operator/query/

https://learn.mongodb.com/learning-paths/mongodb-python-developer-path

Mongo CRUD operations are atomic on the document level (including embedded ones). 
Significa que un cambio debe completarse en su totalidad o no modificar nada en absoluto. (por ejemplo ante un fallo del sistema, una operación no puede quedar a medias)

Create

In [ ]:
#FOR FIRST AND UNIQUE TIME, BUT I BELIVE IS BETTER USE UPSERT, FROM .UPDATE()

db.collectionName.insertOne({field: "value"}) #just one document 

db.collectionName.insertMany([{field: "value"}, {field: "value"}]) #List of documents. Note the []
#i even can insert just one, but the important thing is that .insertMany ALWAYS use [] (index start in 0)

db.collectionName.insert()
#can insert one or many (always use [] if many)
#in the output doesn't show the ID created. But off course it create an ID for the documents inserted  
#need to wait to complete insert ALL the documents to then query ALL of them and search the ID
#in conclusion is not that great. 



In [ ]:
""" Using own ID, not the auto generated one """
db.collectionName.insertMany([{_id: "idName1", field: "value", stock: 100}, {field: "value", _id: "jelow"}]) 

#def the own id with "_id: " parameter

db.collectionName.insertMany([{_id: "idName1", field: "value1"}, {_id: "idName1", field: "value2"}]) 
#if i over ride an _id. Mongo will insert everything before the overrided document. But stops when the error is found.
#as you notice that error may occur in this case. Cuz _id in both documents is the same.

db.collectionName.insertMany([{_id: "idName1", field: "value1"},
        {_id: "idName1", field: "value2"},
        {_id: "idNameOTHER", field: "value2"}], {ordered: False}) 
#if i want to keep inserting the files after the wrong one. OUT the list, declare the {ordered: False} statement



In [ ]:
""" Write concern (Journaling guaranteed)
w: number of instances
w: 1 default

j: journal. It's like a backup if server fails and don't save the inf in the disk.
if j: True  Todo's list saving all the writes commands (write commands: insert or update), reports succesfull write to me
            if the inf has been saved to the journal

client --> mongo server --> storage engine
There's a storage engine: Responsable for writing data onto DISK (permanent saved inf) 
but also in MEMORY (kinda temporal inf, easier to access than disk inf)
 """

db.collectionName.insertOne({field: "value"}, {writeConcern: {w: 1, j: True}}) #conectado con atlas debe ser j: true

db.collectionName.insertMany([{field: "value"}, {field: "value"}], {writeConcern: {w: 1, j: True}}) 

Importing data

In [ ]:
""" Importin an specific .json full of info """
#Locates the path from the .json file in the normal terminal (cmd)
#i.e. C:\Users\USER\Documents\Trabajo\2. MongoDB\Sec6 here i can find tv-shows.json
> mongoimport name.json -d dbName -c collectionName --jsonArray --drop
mongoimport boxoffice.json -d boxOffice -c movieStarts --jsonArray --drop
mongoimport persons.json -d analytics -c persons --jsonArray --drop

mongoimport -u vivi -p 'vivi' -d boxOffice -c movieStarts --file boxoffice.json --jsonArray --drop --authenticationDatabase admin


"""
-d: specify into wich database i want to import the data
-c: specify into wich collection 
--jsonArray: is needed cuz my .json is an array of documents. (Many docs[])
--drop: if the collection already exist, drop and re-add the new one. otherwise it will append
to the existing collection
"""

Read

In [ ]:
db.collectionName.findOne({}) #if no or empty curly braces returns the 1st doc 
db.collectionName.findOne({name: "Search Name"}) #findOne always return the first document that meets the criteria
db.collectionName.find({}) #show the 20 first docs
db.collectionName.find({}).count() #get back the number of items matched in the criteria search

""" I can use operators to make more specific my search
https://www.mongodb.com/docs/v5.0/reference/operator/query/
 """
db.collectionName.find({searchParameter: {}}) #for operators, need the extra {} 
db.collectionName.find({searchParameter: {$nin: ["hola", "adios"]}}) #find all values where the searchParameter not equal to 1 or 2
db.movies.find({weight: {$lte: 80}})

"""Querying embedded fields 
It is important to use <" "> double quotation marks for embedded docs.
It works too in arrays with embedded documents 
"""
# {field: {sub2_field: "value2", sub22_field: {sub3_field: "value", sub33_field: 123}}}
# {field: {sub2_field: "value2", sub22_field: [{sub3_field: "value", sub33_field: 123},{ sub4: "sol", sub44: "luna"}]}}
db.collectionName.find({"field.sub22_field.sub33_field": {$nin: [456, 12]}})
db.movies.find({"network.country.name": "United Kingdom"})


#{field: "value", field2: 123, field3: [56, 345, 12]}
db.collectionName.find({field3: 56}) #this return all the docs where find in field3 the number 53 and the others inside the same list
db.collectionName.find({field3: [56]}) #return just the list filled with that unique 
""" Projections are use to display only the fields i want to see """
# projections: find the docs that match the <field: "value"> criteria but only display field2
db.collectionName.find({field: "value"}, {_id: 0, field2: 1})


""" $slice """
db.collectionName.find({field: "value"}, {field2: 1, field3: {$slice: [<start element>, <fromStartElementHowManyTakes>]}})


""" $size """
# find the docs with a specific field size


""" $or / $nor """
db.collectionName.find({$or: [{fieldXorY: "something"} , {{fieldYorX: 56}}]}) # or: [{this}, {other}]


""" $and / $not """
db.collectionName.find({$and: [{fieldXorY: "value"} , {{fieldYorX: "othervalue"}}]}) # and : [{this}, {this}]


""" $exist / $type"""
db.collectionName.find({field3: {$exists: true, $ne: null}}) #i'm quering for existing (works boolean) and no equal= null the field3
db.collectionName.find({field3: {$type: "number"}}) #searching for field3 with datatype: number
db.collectionName.find({field3: {$type: ["double", "string"]}}) #can be a list of datatypes in that field too


"""$regex (regular expresion):
Selects documents where values match a specified regular expression. NOT only the docs where the value is unique"""
db.collectionName.find({fieldX: {$regex: /"word"/}}) #the <//> will not look for full equality. Just found the docs that contains the "word" and others 
# example in: C:\Users\USER\Documents\Trabajo\2. MongoDB\Sec7\92. regex


In [ ]:
"""Cursors (pointers):
Iterating over query results. As we notice, find() only shows the 20 fisrt querys (0-19).
Cursors allows me to see the next 20 (20-39). But i can even define the number of querys in every iteration
To iterate a cursor manually simply assign the cursor return by the find() method to the var keyword.

Solo lo tiene db.collectionName.find(), es como si en ese cursor se almacenara the query y en vez de mandar al usuario todos los 
documentos que solicito. Va mandando por paquetes. Esto hace eficiente el envío de información y no satura la memoria de la
app del cliente. No es lo mismo mandar 1000 archivos en una sola tanda, que mandar de 20 en 20 (batches of data). :) 
"""
> const nameCursor = db.collectionName.find("query criteria")
> nameCursor.next() #.next() method will show me ONE doc that fits the query criteria
#if i write nameCursor.next() again will show me the THE NEXT doc that fits the criteria and so on.
> nameCursor #if i execute only this gonna show me the same 20 docs as if i only say db.collectionName.find() PUES CLARO




Update

In [ ]:
db.nameCollection.updateOne({"search parameter"}, {$set: {"update field"}})
#i.e. im updating existing fields
> db.users.updateOne({name: "Max"},{$set: {hobbies: [{title: "Sports", frequency: 5}, 
    {title: "Cooking", frequency: 3}, {title: "Hiking", frequency: 1}]}})

db.nameCollection.updateMany({"search parameter"},{$set: {"update field"}})
#i.e  in the docs where find hobbies.title = sports im creating the booleand field called isSporty
# and updating the field frecuency to 2
> db.users.updateMany({"hobies.title": "Sports"}, {$set: {isSporty: true, frecuency: 2}})


"""Increment / Decrement (works for both: one and many)
no es necesario que siempre en el incremento tambien actualizar otro cambio. 
Puedo usar el $inc solito. Pero <OJO!> nunca puedo usar $set e $inc para el mismo campo.

$inc:   1  increment by 1
$inc:   -1 decrement by 1
"""
db.nameCollection.updateMany({"search parameter"}, {$inc: {field: -1} , $set: {"update field"}})
db.nameCollection.updateOne({"search parameter"}, {$inc: {field: 1} , $set: {"update field"}})
#i.e find Manuel, and decrement his age by 1, and now i change his isSporty state
> db.user.updateOne({name: "Manuel"}, {$inc: {age: -1}, $set: {isSporty: false}})
# i.e of ERROR: cannot increment and update the same field at the same time
> db.user.updateOne({name: "Manuel"}, {$inc: {age: 1}, $set: {age: 30}})


""" $min / $max / $mul (works for both: one and many) """
#$min change to a lower value JUST if the actual value is greater than the new one. 
#i. e. Can pass from age:40 to age: 35 BUT NOT from age: 40 to age:43. Pues porque no sería un cambio a min
db.nameCollection.updateOne({"search parameter"}, {$min: {"field: value"}})

#$max do the opossite. change to a higher value
db.nameCollection.updateOne({"search parameter"}, {$max: {"field: value"}})

#$mul MULTIPLY the field by a number i specified
db.nameCollection.updateMany({"search parameter"}, {$mul: {field: multiplo}})
#i.e. In my current document with name: Manuel, the field age: 38. Now, i multiply age by 1.1
#gonna return now that Manuel age is 41.80
> db.user.updateOne({name: "Manuel"}, {$mul: {age: 1.1}})

"""UPSERT
Combination of update and insert, if document doesn't exit, it will be created"""
> db.user.updateOne({find doc}, {$set:{}}, {upsert: true})




""" Delete / Rid fields (works for both: one and many)
Remember is not the same Null != unexisting field.
In this case i'm gonna drop the field. Make it disappear, puff!
"""
db.nameCollection.updateMany({"query condition"}, {$unset: {field: ""}})
#notice something: field: "" means gonna drop in all the docs, the fild specified
#i. e. in this case i'm just gonna drop the field age where the value is null. so the others with age stands the same
db.users.updateMany({isSporty: true}, {$unset: {age: null}})


""" < $elemMatch: >  USEFUL FOR UPDATE AND FIND 
Si tengo doc1 y doc2 que comparten mismos campos. Dentro de un DOC general y quiero que en mis 
coincidencias solo se cumplan las condiciones de busqueda dentro de un solo doc hijo
                """

#i.e i want to find the docs where hobbies are sports with frec gte than 3
> db.collectionName.find({$and: [{"hobbies.title": "sports"},{"hobbies.frec": {$gte: 3}}]})
#returns{name: Juan, age: 30, hobbies: [{title: sports, frec: 2}, {title: cooking, frec: 3}]}
#it's wrong cuz frec in sports is 2. But query search in all the docs and found in another doc that frec:3

#WITH elemMatch busca que se cumplan las condiciones en un mismo documento embebido. No en el doc general
> db.collectionName.find({hobbies: {$elemMatch: {title: "sports", frec: {$gte: 3}}}})
#returns{name: Max, age: 23, hobbies: [{title: sports, frec: 6}, {title: sleeping, frec: 3}]}


In [ ]:
""" Updating MATCHED array elements  
{name: Max, age: 23, hobbies: [{title: sports, frec: 6}, {title: sleeping, frec: 3}]}
actualizar un solo campo de un doc dentro de un array
"""
# < "   1.$.2   "  > 
# < "hobbies.$.highFrec" > 1. dig into hobbies, $. find the element found in the query and then 3. access the highFrec field
# if highFrec don't exist, then, gets added 
# the < $ > placeholder helps to access quickly to the matched array element. In this case hobbies[0]
> db.collectionName.updateMany({hobbies: {$elemMatch:{title: "sports", frec: {$gte: 3}}}}, {$set: {"hobbies.$.highFrec": true}})


""" Updating ALL array elements 
actualizar un campo dentro de todos los docs en el array
"""
#i.e i wanna change the frec in all hobbies from persons greater than 30 years old
# $[] means update all elements inside the array
> db.collectionName.updateMany({age: {$gte: 30}}, {$inc: {"hobbies.$[].frec": -1}})


""" Finding and updating specific fields in the array 
actualizar un campo específico dentro de ciertos docs en el array
"""
# {"1"} filter is just by identify documents. 
# {"2"} update action
# {"3"} second filter gonna identify array elements to apply the update action in {"2"}
# {"1"} and {"3"} don't need to be the same query. 
> ddb.collectionName.updateMany({"1"}, {"2"}, {"3"})
> db.users.updateMany({"hobbies.frec": {$gt: 2}}, {$set: {"hobbies.$[el].goodFrec": true}}, {arrayFilters: ["el.freq": {$gte: 2}]})
# not mandatory to be <el>, can change the name if i want to 


""" Adding / Deleting elements from the array 
 <arrayName> is the array to wich i want to push/add the new element
 """
#Adding
#one new element (could repeat data)
> ddb.collectionName.updateOne({"find doc"}, {$push: {arrayName:{"field": "value", "field2": 34}}}) 
#addToSet: if i try to add a value that's already part of the array, it will not be added again. IT'S A SETTER!
> ddb.collectionName.updateOne({"find doc"}, {$addToSet: {arrayName:{"field": "value", "field2": 34}}}) 
#more than 1 new element 
> ddb.collectionName.updateOne({"find doc"}, {$push: {arrayName:{$each: [{docArray1}, {docArray2}]}}}) 

#Deleting
# i find the doc and then select the one doc or field i wanna pull/ delete inside the array
> ddb.collectionName.updateOne({"find doc"}, {$pull: {arrayName:{"field": "value"}}})

#deleting the last element from the array
> ddb.collectionName.updateOne({"find doc"}, {$pop: {arrayName: 1}}})
#deleting the first element from the array
> ddb.collectionName.updateOne({"find doc"}, {$pop: {arrayName: -1}}})


Delete

In [ ]:
# Delete THE FIRST record that matches
db.collectionName.deleteOne({"query of doc"})

# Delete ALL matching documents
db.collectionName.deleteMany({"query of doc"})

# Delete ALL entries in a collection
db.collectionName.deleteMany({})
db.collectionName.drop()

# Drop database
show dbs
use dbsName
db.dropDatabase()
